### **Data Loading, Storage, and File Formats**

#### **6.1 Reading and Writing Data in Text Format**

##### ***Mechanics of the function***

1. ***Indexing*** -- Can treat one or more columns as the returned Dataframe, and whether to get columns names from the files, arguments you provide or not at all.
2. ***Type inference and data conversion*** -- the user defined value conversions and custom list of missing value markers.
3. ***Data and Time parsing*** -- Includes combining capability, including combining date and time information spread over multiple columns into a single columns in a result.
4. ***Iterating*** -- Support for iterating over chunks of very large files.
5. ***Unclean data issue*** -- Skipping rows or a footer, comments, or other minor things like numeric data with thousands seperated with commas.


In [1]:
import pandas as pd 
import numpy as np

In [2]:
!type "..\data\ex1.csv"

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
# since it is comma-delimited, we can read_csv
df = pd.read_csv("../data/ex1.csv")

In [4]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# a file will not always have a header row.
!type "..\data\ex2.csv"

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [6]:
# options-  1. you can allow pandas to assign a default comlumn name, or you can specify names yourself
pd.read_csv('../data/ex2.csv',header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('../data/ex2.csv',names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# suppose you wanted the message column to be the index of the returned DataFrame.
names = ['a','b','c','d','message']
pd.read_csv('../data/ex2.csv',names = names,index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [9]:
# in the event you want to form a heirarchical index from multiple column, pass a list of column numbers or names:
!type ..\data\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [10]:
parsed = pd.read_csv('../data/csv_mindex.csv',index_col=['key1','key2'])

In [11]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
# in some cases, a table might not have a fixed delimiter, using a whitespace or some other pattern to seperate fields
list(open('../data/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_csv('../data/ex3.txt',sep= r'\s+')

In [14]:
result # here since the number of columns is less than the number of rows, read_csv infers that the first column must be the index in this special case. 

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [15]:
# exploring the additional arguments in the parser function
!type ..\data\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [16]:
pd.read_csv('../data/ex4.csv',skiprows=[0,2,3]) # skips these rows

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [17]:
!type ..\data\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [18]:
result = pd.read_csv('../data/ex5.csv')

In [19]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [21]:
result = pd.read_csv('../data/ex5.csv',na_values=['NULL']) # to consider null values

In [22]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [23]:
# different NA sentinels can be specified for each column in a dict:
sentinels = {'message':['foo','NA'],'something':['two']}
pd.read_csv('../data/ex5.csv',na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


#### ***Reading Text files in Pieces*** 

In [24]:
pd.options.display.max_rows=10

In [25]:
result = pd.read_csv('../data/ex6.csv')

In [26]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [27]:
# if you want a small number of rows
pd.read_csv('../data/ex6.csv',nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [28]:
# you can also specify the chunk size
chunker = pd.read_csv('../data/ex6.csv',chunksize=1000)

In [29]:
chunker 
# the TextFileReader object returned allows you to iterate over the parts of file according to the chunksize.

In [30]:
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot.sort_values(ascending=False)
# TextFilereader is equipped with get_chunk that enales you to read pieces of an arbitrary size

key
E    368
X    364
L    346
O    343
Q    340
    ... 
5    157
2    152
0    151
9    150
1    146
Length: 36, dtype: object

#### ***Writing Data to Text Format*** 

In [31]:
# Data can also be exported to delimited format
data = pd.read_csv('../data/ex5.csv')

In [32]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [33]:
data.to_csv('../data/out.csv')

In [34]:
!type ..\data\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [35]:
import sys

In [36]:
data.to_csv(sys.stdout,sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [37]:
# missing values appear a empty strings. you might want to change them to something more visible
data.to_csv(sys.stdout,sep='|',na_rep='NULL')

|something|a|b|c|d|message
0|one|1|2|3.0|4|NULL
1|two|5|6|NULL|8|world
2|three|9|10|11.0|12|foo


In [38]:
# row and column labels can be disabled
data.to_csv(sys.stdout,index=False,header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [39]:
# write only subset of column
data.to_csv(sys.stdout,index=False,columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [40]:
# Series also has to_csv method
index = np.arange(10,17)
series = pd.Series(np.arange(7),index=index)
series.to_csv('../data/series.csv')

In [41]:
!type ..\data\series.csv

,0
10,0
11,1
12,2
13,3
14,4
15,5
16,6


#### ***Working with Delimited Formats***

In [42]:
!type ..\data\ex7.csv 
# in some cases some manual pricessing may be necessary

"a","b","c"
"1","2","3"
"1","2","3"


In [43]:
# python in built csv module
import csv
f = open('../data/ex7.csv')
reader = csv.reader(f)

In [44]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [45]:
# it is up to you to do the wrangling necessary to put the data in the form you need
with open('../data/ex7.csv') as f:
    lines = list(csv.reader(f))


In [46]:
header,values = lines[0],lines[1:]

In [47]:
header

['a', 'b', 'c']

In [48]:
values

[['1', '2', '3'], ['1', '2', '3']]

In [49]:
# then we can create a dict of data columns 
data_dict = {h:v for h,v in zip(header,zip(*values))}

In [50]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

### ***JSON DATA***

In [51]:
obj = """
{
"name": "Wes",
"cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
"pet": null,
"siblings": [
    {"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
    {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}
]
}
"""

In [52]:
obj

'\n{\n"name": "Wes",\n"cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],\n"pet": null,\n"siblings": [\n    {"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},\n    {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}\n]\n}\n'

In [53]:
import json

In [54]:
result = json.loads(obj)

In [55]:
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [56]:
# dumps converts python object back to json
asjson = json.dumps(result)

In [57]:
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

In [58]:
siblings = pd.DataFrame(result['siblings'],columns=['name','age'])

In [59]:
siblings

,name,age
0,Scott,34
1,Katie,42


In [60]:
# pandas.read_json can directly convert json datasets into a series or dataframe. the assumption while doing this is that each object in json array is a row int he table 
!type ..\data\example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [61]:
data = pd.read_json('../data/example.json')

In [62]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [63]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [64]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### ***XML and HTML: WEB Scrapping***

In [65]:
tabels = pd.read_html('../data/fdic_failed_bank_list.html')

In [66]:
len(tabels)

1

In [67]:
failures = tabels[0]

In [68]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [69]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [70]:
close_timestamps.dt.year.value_counts()

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 15, dtype: int64

#### ***Parsing XML with lxml.objectify***

In [71]:
from lxml import objectify

In [72]:
path = '../data/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [73]:
root.INDICATOR

<Element INDICATOR at 0x2303e8accc0>

In [74]:
data=[]

In [75]:
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ','DESIRED_CHANGE', 'DECIMAL_PLACES']

In [76]:
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag]= child.pyval
    data.append(el_data)

In [77]:
perf = pd.DataFrame(data)

In [78]:
perf

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8
...,...,...,...,...,...,...,...,...,...,...,...,...
643,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,8,Service Indicators,M,%,97.0,,97.0,
644,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,9,Service Indicators,M,%,97.0,,97.0,
645,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,10,Service Indicators,M,%,97.0,,97.0,
646,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,11,Service Indicators,M,%,97.0,,97.0,


### ***Binary Data Formats***

In [83]:
frame = pd.read_csv('../data/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [84]:
frame.to_pickle('../data/frame_pickle')

In [85]:
pd.read_pickle('../data/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### ***Using HDF5 Format***

In [86]:
frame = pd.DataFrame({'a':np.random.randn(100)})

In [87]:
frame

,a
0,0.345935
1,2.233353
2,0.636857
3,-2.282712
4,-1.217015
...,...
95,-1.835466
96,1.832251
97,0.303960
98,2.135893


In [88]:
store = pd.HDFStore('mydata.h5')

In [89]:
store['obj1'] = frame

In [91]:
store['obj1_col'] = frame

In [92]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [93]:
store['obj1']

,a
0,0.345935
1,2.233353
2,0.636857
3,-2.282712
4,-1.217015
...,...
95,-1.835466
96,1.832251
97,0.303960
98,2.135893


In [94]:
# HDF store supports two storage schemas, 'fixed' and 'table'. The latter is generally slower, but it supports query operations using a special syntax:
store.put('obj2',frame,format='table')

In [95]:
store.select('obj2',where=['index >= 10 and index <= 15'])

,a
10,0.864028
11,0.891950
12,1.062454
13,-1.299082
14,-0.366536
15,0.425613


In [96]:
store.close()

#### ***Reading Microsoft Excel Files***

In [98]:
xlsx = pd.ExcelFile('../data/ex1.xlsx')

In [99]:
pd.read_excel(xlsx,'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [100]:
# if you are reading multiple files, then it is faster to create the Excelfile
frame = pd.read_excel('../data/ex1.xlsx','Sheet1')

In [101]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [ ]:
# to write pandas data to excel
